In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

In [ ]:
def accessmap(row):
    if ((float(row['ASA']) ==0) & (float(row['BSA'])==0)):
        val = 'B'
    elif ((float(row['ASA']) >0) & (float(row['BSA'])==0)):
        val = 'A'
    elif ((float(row['ASA']) >0) & (float(row['BSA'])>0)):
        val = 'I'
    return val

In [ ]:
#################################################################################### Accessibility statistics
Scop3P_allSTR=pd.read_csv('Scop3P_PISA_mapped_Dec2020_META_DSSP.txt',sep='\t')   # all Scop3P PISA mapped
#################################################################################### Accessibility statistics

# Scop3P_allSTR=Scop3P_allSTR[(Scop3P_allSTR['ASA']!='-') & (Scop3P_allSTR['RESIDUE'].isin(['SER','SEP','THR',\
#                                                                                         'TPO','TYR','PTR']))]

# Scop3P_allSTR['Accessibility']=Scop3P_allSTR.apply(accessmap,axis=1)
Scop3P_allSTR.columns

In [ ]:
#######################################################################   Accessibility statistics overall
STRtemp=pd.read_csv('STRTEMPall.txt',sep='\t',index_col=None)
print (STRtemp.columns)
STRtemp=STRtemp[["ACC_ID","UP_POS","SP_BEG","SP_END","PDBID","CHAINID","PDBPOS",'PDBchain','UP_RES','PDB_RES']]
STRtemp['fragment']=STRtemp['SP_END']-STRtemp['SP_BEG']

STRtemp=STRtemp[["ACC_ID","PDBID","CHAINID",'fragment','PDBPOS',"UP_POS",'PDBchain','UP_RES','PDB_RES']]

PDBmodlist=pd.read_csv('PDBIDmodifications.txt',sep='\t')
# PDBmodlist=PDBmodlist[PDBmodlist['Modifications']=='unmodified']
STRtemp=STRtemp.merge(PDBmodlist,on=["PDBID","CHAINID"])
# print (STRtemp['length'].value_counts())
pdblen=pd.read_csv('pdblength.txt',sep='\t')
STRtemp=STRtemp.merge(pdblen,on=['PDBID','CHAINID'])


STRtemp=STRtemp.merge(Scop3P_temp,on=['ACC_ID','UP_POS']).drop_duplicates()
print("after merging to PRIDE,only",STRtemp.shape)
print("after merging to PRIDE,only PDB IDs",STRtemp['PDBID'].nunique())
print("after merging to PRIDE,only UP IDs",STRtemp['ACC_ID'].nunique())


STRtemp=STRtemp[STRtemp['length']>=50]
STRtemp=STRtemp[STRtemp['fragment']>=50]

STRtemp=STRtemp.merge(Scop3P_allSTR,on=['PDBID','CHAINID','UP_POS',"ACC_ID"])
print ("after fragment and length filter and merging access data",STRtemp.shape)
print("after fragment and length PDB IDs",STRtemp['PDBID'].nunique())
print("after fragment and length UP IDs",STRtemp['ACC_ID'].nunique())
# STRtemp=STRtemp[STRtemp.Method.isin(['x-ray','em'])]
sns.set(style='darkgrid')

# ############################################################################################### Strict P sites 
# Strictsite=pd.read_csv('P-site_strictG1.txt',sep='\t',index_col=None)
# Strictsite=Strictsite[['ACC_ID','UP_POS']]
# STRtemp=STRtemp.merge(Strictsite,on=['ACC_ID','UP_POS'])
# print (STRtemp.columns)
print ("\n################################################################")
print ("Total sites in filter now",Scop3P[Scop3P['#Proj']>=1].shape[0])
STRtemp=STRtemp[STRtemp['#Proj']>=1]


Scop3P_allSTR_filt=STRtemp

# Scop3P_allSTR_filt=Scop3P_allSTR_filt.merge(Scop3P_temp,on=['ACC_ID','UP_POS']).drop_duplicates()
# print("after strict site filtering",Scop3P_allSTR_filt.shape)
print ("\n############# HERE i take all residues in NMR as well becoz these residues dont have ASA ###########")
Scop3P_allSTR_filt['RESIDUE']= np.where(Scop3P_allSTR_filt['RESIDUE'] == '-', Scop3P_allSTR_filt['UP_RES'],\
                                        Scop3P_allSTR_filt['RESIDUE'])


Scop3P_allSTR_filt['RESIDUE']= np.where(Scop3P_allSTR_filt['RESIDUE'] == 'X', Scop3P_allSTR_filt['PDB_RES'],\
                                        Scop3P_allSTR_filt['RESIDUE'])
Scop3P_allSTR_filt.loc[(Scop3P_allSTR_filt.RESIDUE == 'S'),'RESIDUE']='SER'
Scop3P_allSTR_filt.loc[(Scop3P_allSTR_filt.RESIDUE == 'T'),'RESIDUE']='THR'
Scop3P_allSTR_filt.loc[(Scop3P_allSTR_filt.RESIDUE == 'Y'),'RESIDUE']='TYR'

Scop3P_allSTR_filt=Scop3P_allSTR_filt[Scop3P_allSTR_filt['RESIDUE'].isin(['SER','SEP','THR',\
                                                                                        'TPO','TYR','PTR'])]

print("\nafter strict site filtering and amino acid filtering",Scop3P_allSTR_filt.shape[0])
Scop3P_allSTR_filt.to_csv('TTTTT.txt',sep='\t',index=False)
print("after fragment and length PDB IDs",Scop3P_allSTR_filt['PDBID'].nunique())
print("after fragment and length UP IDs",Scop3P_allSTR_filt['ACC_ID'].nunique())

print ("\n RESIDUES and Ssec")
print (Scop3P_allSTR_filt.RESIDUE.value_counts())
print (Scop3P_allSTR_filt.Ssec.value_counts())
print ('\n')
print (Scop3P_allSTR_filt.RESIDUE.value_counts(normalize=True))
print (Scop3P_allSTR_filt.Ssec.value_counts(normalize=True))
print ("\nSER/SEP in Ssec",Scop3P_allSTR_filt[Scop3P_allSTR_filt['RESIDUE'].isin(['SER','SEP'])].Ssec.value_counts(normalize=True))
print ("\nTHR/TPO in Ssec",Scop3P_allSTR_filt[Scop3P_allSTR_filt['RESIDUE'].isin(['THR','TPO'])].Ssec.value_counts(normalize=True))
print ("\nTYR/PTR in Ssec",Scop3P_allSTR_filt[Scop3P_allSTR_filt['RESIDUE'].isin(['TYR','PTR'])].Ssec.value_counts(normalize=True))

Scop3P_allSTR_filt=Scop3P_allSTR_filt[(Scop3P_allSTR_filt['ASA']!='-') & (Scop3P_allSTR_filt['RESIDUE'].isin(['SER','SEP','THR',\
                                                                                        'TPO','TYR','PTR']))]

Scop3P_allSTR_filt['Accessibility']=Scop3P_allSTR_filt.apply(accessmap,axis=1)
# print(Scop3P_allSTR_filt.columns)
print("\nAfter ASA==- filter",Scop3P_allSTR_filt.shape)

Scop3P_allSTR_filt[['PDBchain']].to_csv('PDBchain_for_NP_access.txt',sep='\t',index=False)

# Scop3P_allSTR_filt.to_csv('TEST1.txt',sep='\t',index=False)

#########################################################################################  Merging PRIDE only sites
Scop3P_allSTR_filt=Scop3P_allSTR_filt.merge(Scop3P_temp,on=['ACC_ID','UP_POS']).drop_duplicates()
print("after merging to PRIDE,only",Scop3P_allSTR_filt.shape)


access=Scop3P_allSTR_filt[Scop3P_allSTR_filt['Accessibility']=='A']
buried=Scop3P_allSTR_filt[Scop3P_allSTR_filt['Accessibility']=='B']
interface=Scop3P_allSTR_filt[Scop3P_allSTR_filt['Accessibility']=='I']


print ("Total number of sites in accessible region", access.shape[0])
print ("Total number of sites in interface region", interface.shape[0])
print ("Total number of sites in buried region", buried.shape[0])

print ("%%%%%%%%%%%%%%%%%%%%%%%% Accessible %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")

plt.figure(figsize=(7,5))
prob_A = access['RESIDUE'].value_counts(normalize=False)
prob_A.plot(kind='bar',color='green',rot=0)
plt.title('Accessible residues absolute count')
plt.tick_params(axis='x', labelsize=12)
print (access['RESIDUE'].value_counts(normalize=False))

print (access['RESIDUE'].value_counts(normalize=True))


print ("%%%%%%%%%%%%%%%%%%%%%%%  Buried  %%%%%%%%%%%%%%%%%%%%%%%%%%%")


plt.figure(figsize=(7,5))
prob_B = buried['RESIDUE'].value_counts(normalize=False)
prob_B.plot(kind='bar',color='steelblue',width=0.3,rot=0)
plt.title('Buried residues absolute count')
plt.tick_params(axis='x', labelsize=12)
print (buried['RESIDUE'].value_counts(normalize=False))

print (buried['RESIDUE'].value_counts(normalize=True))
print ("%%%%%%%%%%%%%%%%%%%%%%%%  Interface  %%%%%%%%%%%%%%%%%%%%%%%%%%%")


plt.figure(figsize=(7,5))
prob_I = interface['RESIDUE'].value_counts(normalize=False)
prob_I.plot(kind='bar',color='orange',rot=0)
plt.title('Interface residues absolute count')
plt.tick_params(axis='x', labelsize=12)
print (interface['RESIDUE'].value_counts(normalize=False))

print (interface['RESIDUE'].value_counts(normalize=True))
print ("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")

plt.figure(figsize=(7,5))

Scop3P_allSTR_filt['Accessibility'].value_counts().plot.bar(color=['green','orange','steelblue'],rot=0)
plt.tick_params(axis='x', labelsize=12)


#######################################################################################################

accessdata=access[['ACC_ID', 'PDBID', 'CHAINID','UP_POS','PDBPOS_y','RESIDUE','ASA', 'BSA','Ssec']]
accessdata.columns=['ACC_ID', 'PDBID', 'CHAINID','UP_POS','PDBPOS','RESIDUE','ASA', 'BSA','Ssec']
accessdata.PDBPOS=accessdata.PDBPOS.astype(int)
accessdata=accessdata.merge(accdataSTY,on=['ACC_ID', 'PDBID', 'CHAINID','PDBPOS'])

interfacedata=interface[['ACC_ID', 'PDBID', 'CHAINID','UP_POS','PDBPOS_y','RESIDUE','ASA', 'BSA','Ssec']]
interfacedata.columns=['ACC_ID', 'PDBID', 'CHAINID','UP_POS','PDBPOS','RESIDUE','ASA', 'BSA','Ssec']
interfacedata.PDBPOS=interfacedata.PDBPOS.astype(int)
interfacedata=interfacedata.merge(accdataSTY,on=['ACC_ID', 'PDBID', 'CHAINID','PDBPOS'])

burieddata=buried[['ACC_ID', 'PDBID', 'CHAINID','UP_POS','PDBPOS_y','RESIDUE','ASA', 'BSA','Ssec']]
burieddata.columns=['ACC_ID', 'PDBID', 'CHAINID','UP_POS','PDBPOS','RESIDUE','ASA', 'BSA','Ssec']
burieddata.PDBPOS=burieddata.PDBPOS.astype(int)
burieddata=burieddata.merge(accdataSTY,on=['ACC_ID', 'PDBID', 'CHAINID','PDBPOS'])

###########################################################################################################
plt.rcParams['axes.labelsize'] = 16
accessdata11_P=accessdata[accessdata['Accesibility'].str.count(',').add(1)==11]
accessP=accessdata11_P[['Accesibility']]
accessP[['P-5','P-4','P-3','P-2','P-1','P','P1','P2','P3','P4','P5']]=accessP.Accesibility.str.split(",",expand=True) 
accessP=accessP.drop(['Accesibility'],axis=1)
accessP=accessP.apply(pd.value_counts, axis=0,normalize=True)
accessP.T.plot(kind='bar',rot=0,stacked=True,legend=False,color=['green','steelblue','orange'],title='Accessible residues',figsize=(7,5)) 
plt.tick_params(axis='x', labelsize=12)

interfacedata11_P=interfacedata[interfacedata['Accesibility'].str.count(',').add(1)==11]
interfaceP=interfacedata11_P[['Accesibility']]
interfaceP[['P-5','P-4','P-3','P-2','P-1','P','P1','P2','P3','P4','P5']]=interfaceP.Accesibility.str.split(",",expand=True) 
interfaceP=interfaceP.drop(['Accesibility'],axis=1)
interfaceP=interfaceP.apply(pd.value_counts, axis=0,normalize=True)
interfaceP.T.plot(kind='bar',rot=0,stacked=True,legend=False,color=['green','steelblue','orange'],title='Accessible residues',figsize=(7,5)) 
plt.tick_params(axis='x', labelsize=12)

burieddata11_P=burieddata[burieddata['Accesibility'].str.count(',').add(1)==11]
buriedP=burieddata11_P[['Accesibility']]
buriedP[['P-5','P-4','P-3','P-2','P-1','P','P1','P2','P3','P4','P5']]=buriedP.Accesibility.str.split(",",expand=True) 
buriedP=buriedP.drop(['Accesibility'],axis=1)
buriedP=buriedP.apply(pd.value_counts, axis=0,normalize=True)
buriedP.T.plot(kind='bar',stacked=True,rot=0,legend=False,color=['green','steelblue','orange'],title='Accessible residues',figsize=(7,5)) 
plt.tick_params(axis='x', labelsize=12)


print ("Access, interface and buried 11 data points",accessdata11_P.shape[0],interfacedata11_P.shape[0],\
      burieddata11_P.shape[0])

In [ ]:
########################################################################### Secondary stcructure geenral all
######################################################## Secondary structure overall and A/B/I is what needed now
# print ("all",Scop3P_allSTR_filt.Ssec.value_counts(normalize=True))
plt.figure(figsize=(7,5))
print (Scop3P_allSTR_filt.Ssec.value_counts(normalize=True))
labels=['C','H','E','-']
Scop3P_allSTR_filt.Ssec.value_counts(normalize=True).plot(kind='pie', fontsize=18,colors=['goldenrod','darkcyan','grey','red'])


# colours = {'C': 'goldenrod',
#            'H': 'darkcyan',
#            'E': 'grey',
#            '-': 'C3'}
fig, ax = plt.subplots(1, 3,figsize=(15,10))
# print ("access",access.Ssec.value_counts(normalize=True))
access.Ssec.value_counts(normalize=True).plot(kind='pie', fontsize=10,ax=ax[0],\
                                              labels=labels,colors=['goldenrod','darkcyan','grey'],title='Access')

# print ("interface",interface.Ssec.value_counts(normalize=True))
interface.Ssec.value_counts(normalize=True).plot(kind='pie', fontsize=10,ax=ax[1],\
                                                labels=labels,colors=['goldenrod','darkcyan','grey'],title='Interface')


# print ("buried",buried.Ssec.value_counts(normalize=True))
buried.Ssec.value_counts(normalize=True).plot(kind='pie', fontsize=10,ax=ax[2],\
                                             labels=labels,colors=['goldenrod','darkcyan','grey'],title='Buried')
# ax[2].tick_params(axis=u'both', which=u'both',length=0)